In [1]:
! pip install transformers==3.0.2

     |████████████████████████████████| 778kB 5.6MB/s 
     |████████████████████████████████| 1.1MB 14.6MB/s 
     |████████████████████████████████| 3.0MB 23.7MB/s 
     |████████████████████████████████| 890kB 40.6MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=367e05deb804fe4dc4b4e1025baafdc2f22a739a5cbe3a52be0d4a5abea9e216
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [2]:
! pip install onnxruntime

     |████████████████████████████████| 3.8MB 5.8MB/s 


In [3]:
!pip install contractions

     |████████████████████████████████| 245kB 4.1MB/s 
     |████████████████████████████████| 317kB 33.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81691 sha256=c2f51c92752460ae673c6817aeb174fa96b160986e676719128fcb72cbe97901
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [4]:
import warnings
warnings.simplefilter('ignore')
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn import metrics
import transformers
import torch
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler
from transformers import DistilBertTokenizer, DistilBertModel
import logging
logging.basicConfig(level=logging.ERROR)

In [5]:
import onnxruntime as ort

In [6]:
MODEL =  ort.InferenceSession('/content/drive/MyDrive/distilbert_emotions_detection.onnx')

In [7]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased', truncation=True, do_lower_case=True)

In [8]:
def to_numpy(tensor):
  if tensor.requires_grad:
    return tensor.detach().cpu().numpy()
  else:
    return tensor.cpu().numpy()

In [10]:
import math
def sigmoid(x):
  return [1 / (1 + math.exp(-xi)) for xi in x]

# **load data set**

In [11]:
df = pd.read_csv('/content/drive/My Drive/isear_processed_emotions.csv')

In [12]:
df = pd.get_dummies(df, columns=['emotions'])

In [13]:
df['labels'] = df[['emotions_anger', 'emotions_fear', 'emotions_joy', 'emotions_sadness']].values.astype('int').tolist()

In [14]:
df = df[(df['emotions_anger']==1) | (df['emotions_fear']==1) | (df['emotions_joy']==1) | (df['emotions_sadness']==1)]

In [15]:
import re
import contractions
import unicodedata
import string
def clean_text(text):
    """
    Function to clean text with basic steps - lower casing, dealing with contractions, remove html codes,
    strip whitespaces, social media cleaning (remove hashtags and URLS), remove punctuationns, using regular expressions.
 
    Parameters
    ----------
    text : str
            Text to be cleaned
    
    Returns
    -------
    text : str
            Cleaned text
    """
    # Lower casing
    text = text.lower()
    
    
    # Remove html codes
    text = re.sub(r"&amp;", " ", text)
    text = re.sub(r"&quot;", " ", text)
    text = re.sub(r"&#39;", " ", text)
    text = re.sub(r"&gt;", " ", text)
    text = re.sub(r"&lt;", " ", text)
    
    # Strips (removes) whitespaces
    text = text.strip(' ')
    
    ################ Social media cleaning ############
    
    # Remove hashtags (Regex @[A-Za-z0-9]+ represents mentions and #[A-Za-z0-9]+ represents hashtags. )
    text = re.sub("(@[A-Za-z0-9]+)|(#[A-Za-z0-9]+)", " ", text)
    
    # Remove URLS (Regex \w+:\/\/\S+ matches all the URLs starting with http:// or https:// and replacing it with space.)
    text = re.sub("(\w+:\/\/\S+)", " ", text)
    text = re.sub(r'http\S+', ' ', text)
    
     # remove old style retweet text "RT"
    text = re.sub(r'^RT[\s]+', '', text)
    # remove accents
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    # remove @users
    text = re.sub(r'@[\w]*', '', text)
    # remove Reddit channel reference /r
    text = re.sub(r'r/', '', text)
    
    # remove reddit username
    text = re.sub(r'u/[\w]*', '', text)
    # remove '&gt;' like notations
    text = re.sub('&\W*\w*\W*;', ' ', text)
    # remove hashtags
    text = re.sub(r'#[\w]*', '', text)
    ###################################################
    
    # Dealing with contractions
    text = contractions.fix(text)
    
    text = re.sub(r"what\'s", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can\'t", "can not ", text)
    text = re.sub(r"n\'t", " not ", text)
    text = re.sub(r"\'t", " not", text )
    text = re.sub(r"i\'m", "i am ", text)
    text = re.sub(r"\'em'", " them ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    
    
    # Removes punctuations
    text = re.sub('['+string.punctuation+']', " ", text)
    
	# Removes non alphanumeric characters
    #text = re.sub('\W', ' ', text)
    
    # Removes non alphabetical characters
    text = re.sub('[^a-zA-Z]+', ' ', text)
    
    # Replaces all whitespaces by 1 whitespace
    text = re.sub('\s+', ' ', text)
    
    return text

In [16]:
df['text'] = df['text'].apply(clean_text)

In [17]:
df.reset_index(drop=True, inplace=True)

In [18]:
df = df[["text", "labels"]]

# **create data loader**

In [19]:
class InferenceDataset(Dataset):
    def __init__(self, texts, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.text = texts
        self.max_len = max_len

    def __len__(self):
        return len(self.text)

    def __getitem__(self, index):
        text = self.text[index]

        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']


        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long)
        }

In [20]:
data_set = InferenceDataset(df['text'], tokenizer, 100)
data_params = {'batch_size': 64,
                'shuffle': False,
                'num_workers': 0
                }
data_loader = DataLoader(data_set, **data_params)

# **predict using dataloader**

In [22]:
pred_dataloader_onnx=[]
for _, data in tqdm(enumerate(data_loader, 0)):
    ids = to_numpy(data['ids'])
    mask = to_numpy(data['mask'])
    onnx_inputs = {'ids': ids, 'mask': mask}
    outputs = MODEL.run(None, onnx_inputs)
    for i in range(outputs[0].shape[0]):
        pred_dataloader_onnx.append(sigmoid(outputs[0][i]))


0it [00:09, ?it/s]


TypeError: ignored

In [ ]:
len(pred_dataloader_onnx)

4381

In [ ]:
pred_y = np.array(pred_dataloader_onnx)
real_y = np.array(df.labels.values.tolist())
for i in range(len(pred_y)):
  for j in range(4):
    pred_y[i, j] =1 if pred_y[i, j] >= 0.5 else 0
pred_y = pred_y.astype('int')

# **previous method: using loop**

In [ ]:
pred = []
from tqdm import tqdm
for text in tqdm(df.text.values):
  input = tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=100,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
  ids = torch.tensor([input['input_ids']], dtype=torch.long)
  mask = torch.tensor([input['attention_mask']], dtype=torch.long)
  token_type_ids = torch.tensor([input["token_type_ids"]], dtype=torch.long)
  onnx_inputs = {
                'ids': to_numpy(ids), 
               'mask': to_numpy(mask),
               }

  output = MODEL.run(None, onnx_inputs)
  pred.append(sigmoid(output[0][0]))







  0%|          | 0/4381 [00:00<?, ?it/s]





  0%|          | 1/4381 [00:00<12:25,  5.87it/s]





  0%|          | 2/4381 [00:00<12:23,  5.89it/s]





  0%|          | 3/4381 [00:00<12:15,  5.95it/s]





  0%|          | 4/4381 [00:00<12:04,  6.04it/s]





  0%|          | 5/4381 [00:00<12:06,  6.02it/s]





  0%|          | 6/4381 [00:01<12:12,  5.97it/s]





  0%|          | 7/4381 [00:01<12:11,  5.98it/s]





  0%|          | 8/4381 [00:01<12:13,  5.96it/s]





  0%|          | 9/4381 [00:01<12:16,  5.94it/s]





  0%|          | 10/4381 [00:01<12:08,  6.00it/s]





  0%|          | 11/4381 [00:01<12:02,  6.05it/s]





  0%|          | 12/4381 [00:01<11:57,  6.09it/s]





  0%|          | 13/4381 [00:02<12:18,  5.92it/s]





  0%|          | 14/4381 [00:02<12:20,  5.90it/s]





  0%|          | 15/4381 [00:02<12:19,  5.91it/s]





  0%|          | 16/4381 [00:02<12:08,  5.99it/s]





  0%|          | 17/4381 [00:02<11:59,  6.07it/s]





  0%|          | 18/43

KeyboardInterrupt: ignored

In [ ]:
pred_y = np.array(pred)
real_y = np.array(df.labels.values.tolist())
for i in range(len(pred_y)):
  for j in range(4):
    pred_y[i, j] =1 if pred_y[i, j] >= 0.5 else 0
pred_y = pred_y.astype('int')

# **overall metrics**

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

In [ ]:
precision_score(real_y, pred_y, average='macro')

0.9423651711484868

In [ ]:
recall_score(real_y, pred_y, average='macro')

0.8424579377071434

In [ ]:
f1_score(real_y, pred_y, average='macro')

0.887377802826699

# **precision score**

In [ ]:
precision = [precision_score(real_y[:, 0], pred_y[:, 0]),
             precision_score(real_y[:, 1], pred_y[:, 1]),
             precision_score(real_y[:, 2], pred_y[:, 2]),
             precision_score(real_y[:, 3], pred_y[:, 3]),
             precision_score(real_y, pred_y, average='macro'),
             precision_score(real_y, pred_y, average='micro'),
             precision_score(real_y, pred_y, average='weighted')]

# **recall score**

In [ ]:
recall = [recall_score(real_y[:, 0], pred_y[:, 0]),
          recall_score(real_y[:, 1], pred_y[:, 1]),
          recall_score(real_y[:, 2], pred_y[:, 2]),
          recall_score(real_y[:, 3], pred_y[:, 3]),
          recall_score(real_y, pred_y, average='macro'),
          recall_score(real_y, pred_y, average='micro'),
          recall_score(real_y, pred_y, average='weighted')]

# **f1 score**

In [ ]:
f1 = [f1_score(real_y[:, 0], pred_y[:, 0]),
      f1_score(real_y[:, 1], pred_y[:, 1]),
      f1_score(real_y[:, 2], pred_y[:, 2]),
      f1_score(real_y[:, 3], pred_y[:, 3]),
      f1_score(real_y, pred_y, average='macro'),
      f1_score(real_y, pred_y, average='micro'),
      f1_score(real_y, pred_y, average='weighted')]

# **overall**

In [ ]:
metric = pd.DataFrame([precision, recall, f1], columns=['anger', 'fear', 'joy', 'sadness', 'macro', 'micro', 'weighted'], index=['precision', 'recall','f1'])

In [ ]:
metric

,anger,fear,joy,sadness,macro,micro,weighted
precision,0.896739,0.970954,0.970060,0.931707,0.942365,0.939664,0.942346
recall,0.903285,0.854795,0.740402,0.871350,0.842458,0.842502,0.842502
f1,0.900000,0.909179,0.839813,0.900519,0.887378,0.888434,0.887395
